# The documents

In [ ]:
laws_of_tertamonia = {
    1: "All citizens have the right to freedom of speech, provided it does not incite violence or spread falsehoods.",    2: "Discrimination based on race, religion, gender, or ethnicity is strictly prohibited in all forms.",
    3: "Every child is entitled to free primary education and access to secondary education.",    4: "Health services must be accessible to all citizens regardless of their financial status.",
    5: "Individuals are presumed innocent until proven guilty in a court of law.",    6: "The privacy of individuals is inviolable; unlawful surveillance or intrusion into personal data is prohibited.",
    7: "Citizens have the right to peaceful assembly and protest without fear of retribution.",    8: "Fair wages must be provided for all forms of labor, with strict enforcement of a minimum wage.",
    9: "Employment of children under the age of 14 in hazardous jobs is prohibited.",    10: "Torture and cruel or inhumane treatment of any individual are strictly forbidden.",
    11: "Marriages must be based on mutual consent of both parties, without coercion.",    12: "Freedom of religion or belief is guaranteed, including the right to practice or not practice any faith.",
    13: "Environmental protection is mandatory; deforestation and pollution are punishable by law.",    14: "All citizens have the right to safe and clean drinking water.",
    15: "Public officials must disclose assets annually to ensure accountability.",    16: "Bribery and corruption in any form are punishable by imprisonment and fines.",
    17: "Weapons possession is regulated, requiring thorough background checks and a valid license.",    18: "All vehicles must adhere to road safety standards, including emissions controls.",
    19: "Tax evasion is a criminal offense with penalties proportional to the amount evaded.",    20: "Unlawful seizure of property is prohibited; citizens have the right to ownership of private property.",
    21: "Animal cruelty is strictly forbidden, with special protections for endangered species.",    22: "Citizens must participate in jury duty when called upon, except in cases of valid exemptions.",
    23: "All individuals have the right to vote in free and fair elections once they reach the age of 18.",
    24: "Slavery or servitude in any form is prohibited.",    25: "Cybercrimes, including hacking and identity theft, carry severe penalties.",
    26: "All businesses must ensure transparency in operations and adhere to consumer protection laws.",
    27: "Human trafficking is a crime with maximum penalties of life imprisonment.",    28: "Involuntary organ or tissue donation is illegal under all circumstances.",
    29: "Citizens have the right to access public records and information for transparency in governance.",    30: "Any form of sexual harassment in workplaces, public spaces, or online is punishable by law.",
    31: "The legal drinking age is set at 21 years, with strict enforcement in public venues.",    32: "All buildings must comply with safety and accessibility standards, especially for individuals with disabilities.",
    33: "Defamation and spreading false information that harms reputations are punishable offenses.",    34: "Landlords are prohibited from evicting tenants without due notice and legal cause.",
    35: "Intellectual property rights are protected to encourage creativity and innovation.",    36: "Human rights education must be included in school curriculums.",
    37: "Emergency response services must be free of charge and available to all.",    38: "Excessive noise pollution in residential areas is prohibited.",
    39: "Unlawful detention without trial is strictly prohibited.",    40: "Non-biodegradable plastic products are banned to promote sustainability.",
    41: "All employers must provide paid parental leave for both parents after childbirth or adoption.",    42: "Citizens have the right to cross-examine witnesses in legal trials.",
    43: "Accurate labeling of all consumer goods is mandatory to ensure informed choices.",    44: "Freedom of the press is guaranteed, with protections for journalists against censorship.",
    45: "Drug trafficking and possession of illegal substances carry severe penalties.",    46: "Citizens have the right to access free public libraries in every region.",
    47: "Exploitation of workers, including unpaid overtime, is illegal.",    48: "All refugees and asylum seekers must be treated with dignity and provided fair hearings.",
    49: "Disposal of toxic waste without proper treatment is a criminal offense.",    50: "Elected representatives must hold public consultations before passing major laws."
}


# Setting up the API key for gemini

In [ ]:
import google.generativeai as genai
genai.configure(api_key='YOUR_API_KEY')

# Embedding the documents

In [ ]:
import faiss
import numpy as np
def get_embedding(text):
    embedding_result = genai.embed_content(
        model='models/embedding-001',
        content=text
    )
    if 'embedding' in embedding_result:
        embedding_vector = np.array(embedding_result['embedding'], dtype=np.float32)
        return embedding_vector
    else:
        raise ValueError(f"No 'embedding' field found for document: {text}")

embeddings = []
for key,value in laws_of_tertamonia.items():
    # print (key,value)
    holder = get_embedding(value)
    embeddings.append(holder)



# Consultation

In [21]:
print(f"Number of embedded documents: {len(embeddings)}")
print(f"Embedding dimension: {len(embeddings[0])}")

Number of embedded documents: 50
Embedding dimension: 768


# Saving the embeddings

In [22]:
import os
import json

# Convert embeddings to numpy array
embedded_docs_np = np.array(embeddings, dtype=np.float32)

# Normalize vectors for cosine similarity
faiss.normalize_L2(embedded_docs_np)

# Create FAISS index
embedding_dim = embedded_docs_np.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(embedded_docs_np)

# Create save directory
save_folder = "rag_system"
os.makedirs(save_folder, exist_ok=True)

# Save FAISS index
index_path = os.path.join(save_folder, "index.faiss")
faiss.write_index(index, index_path)

# Saving the documents

In [ ]:
# Save documents mapping
docs_mapping = {
    str(key): {
        "law": value,
    }
    for key,value in laws_of_tertamonia.items()
}

with open(os.path.join(save_folder, "documents.json"), "w") as f:
    json.dump(docs_mapping, f)